In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from simsity.service import Service 
from simsity.datasets import fetch_clinc
from simsity.indexer import PyNNDescentIndexer
from simsity.preprocessing import ColumnLister

from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer

# We first fetch a dataset
df = fetch_clinc()

# The Encoder handles the encoding of the datapoints
encoder = make_pipeline(ColumnLister(column="text"), CountVectorizer())

# The Indexer handles the nearest neighbor search
indexer = PyNNDescentIndexer(metric="euclidean", n_jobs=6)

# The Service combines them together into a service
service = Service(
    indexer=indexer,
    encoder=encoder
)

# Important for later: we're only passing the 'text' column to encode
service.train_from_dataf(df, features=["text"])

/home/vincent/Development/simwity/venv/lib/python3.7/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [3]:
# Query the datapoints
# Note that the keyword argument here refers to 'text'-column
service.query(text="please set a timer", n_neighbors=5, out="dataframe")

,text,dist
0,set a timer,1.000000
1,please set a timer for,1.000000
2,"""please set 5 minute timer",1.000000
3,please set a 4 minute timer,1.000000
4,start a timer please,1.414214


In [5]:
import ipywidgets as widgets

def reduce(q):
    subset = service.query(text=q, n_neighbors=15, out="dataframe")
    display(subset)

q = widgets.Text()

out = widgets.interactive_output(reduce, {'q': q})

widgets.VBox([q, out])